In [1]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import curve_fit


In [2]:
def sabr_implied_volatility(fwd_price, strike, time_to_expiry, alpha, beta, rho, nu, full=False):

#    Calculate SABR implied volatility using the approximation formula with the higher order term.
    
#    fwd_price : Forward price of the underlying asset
#    strike : Strike price
#    time_to_expiry : Time to maturity (in years)
#    alpha : Volatility of volatility (vol-of-vol)
#    beta : Parameter that defines skew (typically between 0 and 1)
#    rho : Correlation between the asset price and volatility
#    nu : Initial volatility
#    return : SABR implied volatility
        
    #one minus beta term
    oneMinusBeta = 1 - beta
    
    # First term: main term (F * K scaling)
    main_denominator = (fwd_price * strike)**(oneMinusBeta / 2)
    
    # Second term: volatility of volatility correction
    volatility_of_volatility_term = (oneMinusBeta**2 / 24) * \
                                    (alpha**2 / (fwd_price * strike)) * \
                                     time_to_expiry
    
    # Third term: correlation adjustment
    correlation_term = ((rho * beta * nu) / 4) * \
                       ((fwd_price / strike)**(oneMinusBeta / 2))
    
    # Fourth term: higher-order correction with the 1920 term
    if full:
        higher_order_term = ((2 - 3 * rho**2) / 1920) * \
                             (alpha**4 / ((fwd_price * strike)**2)) * \
                              time_to_expiry**2
    else:
        higher_order_term = 0
    
    # Combine all terms to compute the implied volatility
    implied_vol = (alpha / main_denominator) * \
                  (1 + volatility_of_volatility_term + correlation_term + higher_order_term)
    
    return implied_vol


In [3]:
def sabr_objective(alphaRhoNu, forward_price, strikes, time_to_expiry, volatilities, beta, full=False):
    alpha, rho, nu = alphaRhoNu
    sabr_implied_volatility_vectored = np.vectorize(sabr_implied_volatility)
    model_vols = np.array([sabr_implied_volatility_vectored(forward_price, strikes, time_to_expiry, 
                                                            alpha, beta, rho, nu, full) for K in strikes])
    error = model_vols - volatilities
    return np.sum(error**2)

#result = minimize(objective, initial_guess, args=(strikes, observed_vols, forward_price, alpha),
#                  bounds=bounds, method='L-BFGS-B')
    

In [4]:
def calcWowAlphVols(atmVol, wow, convexity, alpha):
    
    hiVol = atmVol * (1 + wow + (convexity * -alpha))
    loVol = atmVol * (1 - wow + (convexity * -alpha))
    
    return hiVol, loVol


def calcWowAlphPrice(fwdPrice, strikePrice, timeToExp, putCall, atmVol, alpha, wow, convexity):
    
    hiVol = atmVol * (1 + wow + (convexity * -alpha))
    loVol = atmVol * (1 - wow + (convexity * -alpha))
    
    pFunc = np.vectorize(calcBSPrice)
    hiVolPrice = pFunc(hiVol, fwdPrice, strikePrice, timeToExp, putCall)
    loVolPrice = pFunc(loVol, fwdPrice, strikePrice, timeToExp, putCall)
    
    wowAlphaPrice = (hiVolPrice + loVolPrice) / 2
    
    return wowAlphaPrice

    
def calcWowAlphPriceVectors(indepVars, wow, convexity):

    fwdPrice, strikePrice, timeToExp, atmVol, alpha = indepVars
     
    putCall = makePutCall(fwdPrice, strikePrice)
    
    wowAlphaPrice = calcWowAlphPrice(fwdPrice, strikePrice, timeToExp, putCall, atmVol, alpha, wow, convexity)
        
    return wowAlphaPrice
